In [1]:
def global_alignment(s1, s2):
    n, m = len(s1), len(s2)
    
    nlines = n + 1
    ncols = m + 1
    
    matrix = [ [float('-inf') for _ in range(ncols)] for __ in range(nlines)]
    ancestors = [[0 for _ in range(ncols)] for __ in range(nlines)]
    
    for i in range(ncols):
        matrix[0][i] = -i
        ancestors[0][i] = 'left'
    
    for i in range(nlines):
        matrix[i][0] = -i
        ancestors[i][0] = 'up'
    
    for i in range(1, nlines):
        for j in range(1, ncols):
            mismatch = 1 if s1[i-1] == s2[j-1] else -1
                
            diag = matrix[i-1][j-1] + mismatch
            up = matrix[i-1][j] - 1
            left = matrix[i][j-1] - 1
            best = max(diag, up, left)
            
            matrix[i][j] = best
            
            if best == diag:
                ancestors[i][j] = 'diag'
            elif best == up:
                ancestors[i][j] = 'up'
            else:
                ancestors[i][j] = 'left'
        
    line1 = []
    line2 = [] 
    i, j = n, m
    diag_counter = 0
    while i > 0 or j > 0:
        if ancestors[i][j] == 'diag':
            diag_counter += 1
            line1.append(s1[i-1])
            line2.append(s2[j-1])
            i -= 1
            j -= 1
        elif ancestors[i][j] == 'up':
            line1.append(s1[i-1])
            line2.append('_')
            i -= 1
        else:
            line1.append('_')
            line2.append(s2[j-1])
            j -= 1    
        
    line1 = ''.join(line1[::-1])
    line2 = ''.join(line2[::-1])
    return line1 + '\n' + line2

In [16]:
def local_alignment(s1, s2):
    '''И несовпадение, и gap штрафуются единицей'''
    
    n, m = len(s1), len(s2)
    nlines = n + 1
    ncols = m + 1
    
    matrix = [ [0 for _ in range(ncols)] for __ in range(nlines) ]
    ancestors = [ [0 for _ in range(ncols)] for __ in range(nlines) ]
    
    for j in range(ncols):
        matrix[0][j] = 0
    for i in range(nlines):
        matrix[i][0] = 0
     
    # запомнить максимальный элемент
    max_value = float('-inf')
    max_i, max_j = 0, 0
    
    for i, symb1 in enumerate(s1):
        for j, symb2 in enumerate(s2):
            
            mismatch = 1 if symb1 == symb2 else -1
            
            diag = matrix[i][j] + mismatch
            up = matrix[i][j+1] - 1
            left = matrix[i+1][j] - 1
            
            best = max(diag, up, left, 0)
            
            if best > max_value:
                max_value = best
                max_i, max_j = i + 1, j + 1
            
            matrix[i+1][j+1] = best
            
            if best == diag:
                ancestors[i+1][j+1] = 'diag'
            elif best == up:
                ancestors[i+1][j+1] = 'up'
            else:
                ancestors[i+1][j+1] = 'left'
    
    line1, line2 = [], []
    
    cur_element = matrix[max_i][max_j]
    cur_i, cur_j = max_i, max_j  
    while cur_element  > 0:
        taken_path = ancestors[cur_i][cur_j]
        if taken_path == 'diag':
            line1.append(s1[cur_i - 1])
            line2.append(s2[cur_j - 1])
            
            cur_i -= 1
            cur_j -= 1
        elif taken_path == 'up':
            line1.append(s1[cur_i - 1])
            line2.append('-')
            
            cur_i -=1
        else:
            line1.append('-')
            line2.append(s2[cur_j - 1])
            
            cur_j -= 1    
        cur_element = matrix[cur_i][cur_j]
        
    # дописываем по краям что осталось
    # сейчас cur_i, cur_j равны числу оставшихся элементов слева от обнаруженного выравнивания
    i = cur_i - 1  
    j = cur_j - 1
    
    while i >=  0 or j >= 0:
        line1.append(s1[i] if i >= 0 else '-')
        line2.append(s2[j] if j >= 0 else '-')
        i -= 1
        j -= 1
        
    line1 = line1[::-1]
    line2 = line2[::-1]
    
    # элементы на позициях правее max_i - 1 и max_j - 1 тоже не попали в выравнивание 
    i, j = max_i, max_j 
    while i < n or j < m:
        line1.append(s1[i] if i < n else '-')
        line2.append(s2[j] if j < m else '-')
        i += 1
        j += 1
    line1 = ''.join(line1)
    line2 = ''.join(line2)
    return line1 + '\n' + line2

In [14]:
s1 = 'agcagaccgaactttaactttgcattcggcataccg'
s2 = 'cccttcaccactcccataacaccccctataaatatcactc'
a1 = global_alignment(s1, s2)
a2 = local_alignment(s1, s2)
print('Глобальное выравнивание:')
print(a1)
print('*' * 10)
print('Локальное выравнивание:')
print(a2)

Глобальное выравнивание:
agc_agaccgaact___ttaactttgcat_tcggcat_ac_cg
cccttcacc__actcccataacaccccctataaatatcactc_
**********
Локальное выравнивание:
--------------agcagaccgaactttaactttgcattcggcataccg
cccttcaccactcccataacaccccctataaatat-cactc---------


In [29]:
from random import randint
tests = []
letter = {
    1: 'A',
    2: 'T',
    3: 'G',
    4: 'C',
    5: '-'
}
for _ in range(30):
    s1, s2 = '', ''
    for i in range(100):
        n1 = randint(1, 5)
        n2 = randint(1, 5)
        s1 += letter[n1]
        s2 += letter[n2]
    test = (s1, s2)
    tests.append(test)

for test in tests:
    s1, s2 = test
    local_alignment(s1, s2)
    global_alignment(s1, s2)